# Pre-processing read alignments

**Summary:**

This notebook compiles the scripts that are used as part of the pre-processing step of our analysis in order to produce a data table (referred in other notebooks as **dataset**) used accross all downstream analysis. 

It is organized in the following sections:<br>

- **1. Extraction of genomic informations:**<br>
Scripts used for extracting genomic features, namely genomic coordinates of the read alignment and chromosome.  

- **2. Extraction of transcriptomic informations:**<br>
Scripts used for extracting transcriptomics features such as cDNA start & end positions, gene and isoform on which the read is mapped, read orientation, length of the different regions (5' and 3' soft-clips, aligned portion of the sequence, etc). 

- **3. Generation of a summary dataframe:**<br>
Regroupment of genomic and transcriptomic informations inside a single dataframe.<br> 
Also features a script for correcting genomic start & end positions based on transcriptomic alignments.

- **4. Strand Switching Primer (SSP) search:**<br>
Method used for identifying SSP sequences in reads

- **5. Spliced Leader (SL) search:**<br>
Method used for identifying splice leader sequences in reads (see notebooks for **Figure 2 and Figure 3**)

- **6. Endogenous Hairpin search:**<br>
Method used for identifying hairpin structure (see notebook for **Figure 4**)

- **7. Result of Sequence Search:**<br>
Method used for accepting and measuring SL, hairpin, Unidentified and SSP reads propensity at each gene's position
(see notebooks for **Figure 5 and Figure 6**)

---
<br>



## Import libraries

In [ ]:
# datasets & scientific libraries
import pandas as pd
import numpy as np

# json format library
import json

# regex library
import re
from re import search

# plotting libraries
import seaborn as sns
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import MaxNLocator, MultipleLocator, AutoMinorLocator

# sequence manipulation libraries
import pysam
import pyfastx
import parasail
import pyranges as pr
from Bio.Seq import Seq
from Bio import SeqIO

# multiprocessing on all CPU cores
from multiprocessing import Pool

## Set environmental constants

In [ ]:
# Set path where files are stored
path = '/Volumes/elegans/rna_sequencing'

In [ ]:
# Set runs ID
runs = ['SSP_1', 'SSP_2', 'SSP_3', 'SSP_4', 'SSP_5', 'SSP_6', 'SL1_1', 'NP_1', 'NP_2', 'NP_3', 'NP_4', 'NP_5']

---

<br>

# 1. Extract informations from genomic alignments

In [ ]:
# Extract features from genomic alignments (read name, chromosome, genomic start & end positions)

def genomic_stats(input_file, output_file):

    name = []
    start = []
    end = []
    chrom = []

    alignment = pysam.AlignmentFile(input_file, 'rb')

    for read in alignment:

        # Only take into account primary reads
        if not read.is_supplementary and not read.is_secondary and not read.is_unmapped and read.seq is not None:

            name.append(read.query_name)
            chrom.append(alignment.get_reference_name(read.reference_id))
            start.append(read.reference_start+1) # 0-based coordinates - see doc
            end.append(read.reference_end) # Points to one past last coordinate (so 1-based ?)

    # Build dataframe with lists
    table = pd.DataFrame(dict(read=name, chromosome=chrom, genomic_start=start, genomic_end=end))

    # Save dataframe as a .tsv file
    table.to_csv(output_file, sep='\t', index=None)
    
    alignment.close()


In [ ]:
# Handler function
def genomic_stats_handler(ID):
    
    input_file = f'{path}/{ID}/{ID}-genome_sorted.bam'
    output_name = f'{path}/{ID}-genomic_stats.tsv'
    
    # run script
    genomic_stats(input_file, output_name)
    
    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(genomic_stats_handler, runs)

<br>

# 2. Extract informations from transcriptomic alignments

In [ ]:
# Convert isoform name into gene name
# ex: MTCE.35.1 -> MTCE.35

def isoform_to_gene(isoform):
    
    match = re.search(r"\w+.\d+", isoform)

    if match is not None:
        return match.group(0)
    else:
        return None

In [ ]:
# Extract features from transcriptomic alignments (read name, isoform, gene, transcriptomic start & end positions, etc.)

def transcriptomic_stats(input_file, output_file):

    rows = []
    
    transcriptome = pysam.AlignmentFile(input_file, 'rb')

    for read in transcriptome:

        # Only take into account primary reads
        if not read.is_unmapped and not read.is_secondary and not read.is_supplementary and read.seq is not None:

            name = read.query_name
            isoform = transcriptome.get_reference_name(read.reference_id)
            
            # start & end positions relative to the transcript of reference
            transcript_start = read.reference_start+1
            transcript_end = read.reference_end

            # Find read orientation
            orientation = 'antisense' if read.is_reverse else 'sense'

            # sequence length
            seq = read.seq
            seq_length = len(str(seq))
            
            # alignment length
            aln_start = read.query_alignment_start
            aln_end = read.query_alignment_end
            aln_length = len(seq[aln_start:aln_end])

            # start is 0 so soft-clip is rannging from [-n to 0] with n being the soft-clip length
            # which equals to aln start position
            
            # softclips length
            # start is 0 so soft-clip is rannging from [-n to 0] with n being the soft-clip length
            # which equals to aln start position
            five_prime_sc = -aln_start
            three_prime_sc = len(seq[aln_start:])

            # Determine if long or short 5' soft-clip
            softclip = 1 if aln_start > 80 else 0

            # format as dict
            read_row = {'read':name, 'isoform':isoform, 'read_orientation':orientation, 'softclip':softclip,
                        'transcriptomic_start':transcript_start, 'transcriptomic_end':transcript_end,
                        'sequence_length':seq_length, 'alignment_length':aln_length,
                        'SC5':five_prime_sc, 'alignment_start':aln_start, 'alignment_end':aln_end, 'SC3':three_prime_sc}

            rows.append(read_row)
            
    # Generate dataframe
    dataframe = pd.DataFrame(rows)

    # Find gene_ID with isoform value
    dataframe['gene'] = dataframe['isoform'].apply(isoform_to_gene)

    # Set columns order
    dataframe = dataframe[['read', 'gene', 'isoform', 'read_orientation', 'softclip', 'transcriptomic_start', 'transcriptomic_end',
                           'sequence_length', 'alignment_length', 'SC5', 'alignment_start', 'alignment_end', 'SC3']]
    
    # Save dataframe
    dataframe.to_csv(output_file, sep='\t', index=None)



In [ ]:
# Handler function
def transcriptomic_stats_handler(ID):

    input_file = f'{path}/{ID}/{ID}-transcriptome_sorted.bam'
    output_file = f'{path}/{ID}-transcriptome_stats.tsv'

    # run script
    transcriptomic_stats(input_file, output_file)
    
    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(transcriptomic_stats_handler, runs)

<br>

# 3. Generation of a summary dataframe

## 3.a. Parsing genomic and transcriptomic informations

In [ ]:
# set empty list to store intermediary dataframes
dflist = []

# loop over runs
runs = ['SSP_1', 'SSP_2', 'SSP_3', 'SSP_4', 'SSP_5', 'SSP_6', 'SL1_1', 'NP_1', 'NP_2', 'NP_3', 'NP_4', 'NP_5']

for ID in runs:

    genome = f'{path}/{ID}-genomic_stats.tsv'
    transcriptome = f'{path}/{ID}-transcriptome_stats.tsv'

    # open genomic stats
    genome_stats = pd.read_csv(genome, sep='\t')
    genome_stats = genome_stats.set_index('read')

    # open transcriptomics stats
    transcriptome_stats = pd.read_csv(transcriptome, sep='\t')
    transcriptome_stats = transcriptome_stats.set_index('read')

    # concat tables together
    final = pd.concat([genome_stats, transcriptome_stats], join='outer', axis=1)
    
    # only keep reads that are mapped against both genome and transcriptome files
    final = final.loc[(final['genomic_start'].notnull()) & (final['transcriptomic_start'].notnull())]

    final.index.name = 'read'
    final = final.reset_index()

    final['run'] = ID

    final = final[['read', 'gene', 'isoform', 'chromosome', 'read_orientation', 'softclip', 'run',
                   'genomic_start', 'genomic_end', 'transcriptomic_start', 'transcriptomic_end', 
                   'sequence_length', 'alignment_length', 'SC5', 'alignment_start', 'alignment_end', 'SC3']]
    
    dflist.append(final)

stats = pd.concat(dflist, ignore_index=True)


In [ ]:
# Build dictionnary of gene orientation 
strand = pd.read_csv(f'{path}/ref/gene&strand.tsv', sep='\t')
strands = {x['gene']: x['strand'] for idx, x in strand.iterrows()}

# set gene orientation
stats['gene_orientation'] = stats.apply(lambda x: strands[x['gene']], axis=1)

# Set Start and end correctly
stats['start'] = stats.apply(lambda x: x['genomic_start'] if strands[x['gene']] == '+' else x['genomic_end'], axis=1)
stats['end'] = stats.apply(lambda x: x['genomic_end'] if strands[x['gene']] == '+' else x['genomic_start'], axis=1)

stats['genomic_start'] = stats['start']
stats['genomic_end'] = stats['end']

# reorder and drop 'start' and 'end' columns
stats = stats[['read', 'gene', 'isoform', 'chromosome', 'gene_orientation', 'read_orientation', 'softclip', 'run',
               'genomic_start', 'genomic_end', 'transcriptomic_start', 'transcriptomic_end', 
               'sequence_length', 'alignment_length', 'SC5', 'alignment_start', 'alignment_end', 'SC3']]

# save result
stats.to_csv(f'{path}/dataset.tsv', sep='\t', index=None)

<br>

## 3.b. Correction of genomic coordinates based on transcriptome alignments

In [ ]:
# Open GTF file as PyRanges-object
gtf = pr.read_gtf(f'{path}/ref/c_elegans.PRJNA13758.WS270.canonical_geneset.gtf')
gtf = gtf.df

strand = gtf.loc[gtf['Feature'] == 'transcript'].set_index('transcript_id')['Strand'].to_dict()

gtf = gtf.loc[~gtf['exon_id'].isna()][['exon_id', 'Start', 'End']].set_index('exon_id')[['Start', 'End']].to_dict()

# Open transcriptome_relative_coordinates
with open(f'{path}/ref/transcript_exons.json', 'r') as dico:
    exonics_positions = json.loads(dico.read())

In [ ]:
# This script converts a transcriptomic position into a genomic position based on a given isoform
def transcriptome_based_correction(transcript, transcriptome_position):
    
    exons = exonics_positions[transcript]

    for exon, i in exons.items():

        if i[0] <= transcriptome_position <= i[1]:

            delta = transcriptome_position - i[0]

            if strand[transcript] == '+':
                exon_start = gtf['Start'][f'{transcript}.e{exon}']
                genomic_position = exon_start + delta
            else:
                exon_start = gtf['End'][f'{transcript}.e{exon}']
                genomic_position = exon_start - delta

            return genomic_position

        else:
            if int(exon) < len(exons):
                continue
            else:
                return np.NaN

In [ ]:
# Transcriptomic alignment positions are used to refine genomic positions
def correct_genomic_position(input_file):

    table = pd.read_csv(input_file, sep='\t')
    
    # correct start and end positons
    table['corrected_genomic_start'] = table.apply(lambda x: transcriptome_based_correction(x['isoform'], x['transcriptomic_start']), axis=1)
    table['corrected_genomic_end'] = table.apply(lambda x: transcriptome_based_correction(x['isoform'], x['transcriptomic_end']), axis=1)
    
    # useful ?
    table = table.loc[table['gene'].notnull()]
    
    # reorder columns
    final = table[['read', 'gene', 'isoform', 'chromosome', 'gene_orientation', 'read_orientation', 'softclip', 'run',
                   'corrected_genomic_start', 'corrected_genomic_end', 'genomic_start', 'genomic_end', 
                   'transcriptomic_start', 'transcriptomic_end', 
                   'sequence_length', 'alignment_length', 'SC5', 'alignment_start', 'alignment_end', 'SC3']]
    
    # return table with corrected positions
    return final

In [ ]:
# Run script
corrected_dataset = correct_genomic_position(input_file=f'dataset.tsv')

# clean up columns type
col_types = {'read':object, 'gene':object, 'isoform':object, 'chromosome':object,
             'genomic_start':float, 'genomic_end':float, 'corrected_genomic_start':float, 'corrected_genomic_end':float,
             'read_orientation':object, 'softclip':int, 'run':object}

corrected_dataset = corrected_dataset.astype(col_types)

# save table
corrected_dataset.to_csv(f'{path}/dataset.tsv', sep='\t', index=None)

---

<br>

# 4. Sequence search: Strand Switching Primer (SSP)

## 4.a. Extraction of the full 5'soft-clip sequence

In [ ]:
def extract_full_softclip(input_file, output_file, ALN=0):
    
    # create output file
    with open(output_file, 'w+') as file:
    
        # alignments
        alignments = pysam.AlignmentFile(input_file, 'rb')

        for read in alignments:

            if not read.is_unmapped and not read.is_supplementary and not read.is_secondary and read.seq is not None:

                name = read.query_name

                start = read.query_alignment_start
                seq = str(read.seq[0:(start+ALN)])

                # write to file
                file.write(f'>{name}\n{seq}\n')

In [ ]:
# Handler function
def extract_full_softclip_handler(ID):
    
    input_file = f'{path}/{ID}/{ID}-transcriptome_sorted.bam'
    output_file= f'{path}/{ID}/{ID}-five_prime_softclip[SC=FULL|ALN=80].fa'

    # run script
    extract_full_softclip(input_file, output_file, ALN=80)

    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(extract_full_softclip_handler, runs)

## 4.b. SSP sequence search on 5' soft-clip sequences

In [ ]:
SSP_ALIGN_PARAMS = {'match': 1,
                    'mismatch': 0,
                    'gap_open': 2,    # penalty to create a gap
                    'gap_extend': 1}  # penalty to extend a gap (must have created before)

In [ ]:
def semi_global_alignment(reference, query, params):

    subs_mat = parasail.matrix_create("ACGT", params['match'], params['mismatch'])
    alignment = parasail.sg_trace_striped_32(reference, query, params['gap_open'], params['gap_extend'], subs_mat)

    return alignment

In [ ]:
def evaluate_alignment(sequence, motif, seqtype, sensitivity=0.7):
    
    ref = len(motif)
    
    # initialize
    aln_score = None
    aln_size = None
    aln_dist = None
    
    # initial alignment (full seq)
    aln = semi_global_alignment(sequence, motif, params=SSP_ALIGN_PARAMS)
    score = aln.score
    
    # shorter alignments
    if score < len(motif) * sensitivity:
        
        for pos in reversed(range(15, len(motif), 1)):
            
            if seqtype == "5SC":
                aln = semi_global_alignment(sequence, motif[-pos:], params=SSP_ALIGN_PARAMS)
            elif seqtype == "3SC":
                aln = semi_global_alignment(sequence, motif[:pos], params=SSP_ALIGN_PARAMS)
            else:
                break
                
            score = aln.score

            if score < sensitivity * pos:
                continue
            else:
                aln_score = score
                aln_size = pos
                aln_dist = len(sequence) - (int(aln.end_query) + 1)
                break

    else:
        aln_score = score
        aln_size = len(motif)
        aln_dist = len(sequence) - (int(aln.end_query) + 1)
        
    return aln_score, aln_size, aln_dist

In [ ]:
def search_SSP(input_file, output_file, offset=0):
    
    reads = []
    SSP_scores = []
    SSP_sizes = []
    SSP_distances = []
    
    for read in SeqIO.parse(input_file, "fasta"):
        
        name = read.id
        SC5_sequence = str(read.seq)
        
        score, size, dist = evaluate_alignment(SC5_sequence, 'TTTCTGTTGGTGCTGATATTGCTGGG', '5SC')

        if score is None:
            reads.append(name)
            SSP_scores.append(0)
            SSP_sizes.append(0)
            SSP_distances.append(None)
        else:
            reads.append(name)
            SSP_scores.append(score)
            SSP_sizes.append(size)
            SSP_distances.append(-(dist-offset))

    data = pd.DataFrame(dict(read=reads, SSP_score=SSP_scores, SSP_size=SSP_sizes, SSP_dist=SSP_distances))
    data = data.set_index('read')
    data.to_csv(output_file, sep='\t', index=True)

In [ ]:
def search_SSP_handler(ID):
    
    input_file = f'{path}/{ID}/{ID}-five_prime_softclip[SC=FULL|ALN=80].fa'
    output_file= f'{path}/{ID}/{ID}-SSP_search.tsv'

    # run script
    search_SSP(input_file, output_file, offset=80)

    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(search_SSP_handler, runs)

## 4.c. Processing SSP search result

In [ ]:
df_list = []

for ID in runs:
    
    # open SL search result as dataframe
    df = pd.read_csv(f'{path}/{ID}/{ID}-SSP_search.tsv', sep='\t')
    
    # append to new dataframe
    df_list.append(df)
    
# concat dataframes
ssp_result = pd.concat(df_list, axis=0)

In [ ]:
# Merge SL result dataframe with dataset
dataset_SSP = corrected_dataset.merge(ssp_result, on='read', how='left')

In [ ]:
# Determine sequences found based on scores
dataset_SSP['SSP_FOUND'] = dataset_SSP.apply(lambda x: 'FOUND' if x['SSP_dist']==x['SSP_dist'] and x['SSP_dist']-x['SC5']<=80 else 'NOT FOUND', axis=1)


In [ ]:
# Save result
dataset_SSP.to_csv(f'{path}/dataset_+SSP.tsv', sep='\t', index=None)

---

<br>

# 5. Sequence search: Splice Leader (SL)

This section of the notebook corresponds to the code used for searching splice leader (SL) sequences in the different reads obtained from the sequencing of *C. elegans* transcriptome.

It is composed of three sub-sections:<br>
**a.** Extraction of the last 100bp of the 5' soft-clip sequence.<br>
**b.** Definition of the algorithm used for identifying a SL sequence.<br>
**c.** Inclusion of the SL search result to the dataset table.

## 5.a. Extracting 5' soft-clip sequences

In [ ]:
# Extract last 100bp of 5' soft-clip sequence and first 20bp of primary alignment from transcriptomic alignments

def extract_softclip_end(input_file, output_file, SC, ALN=0):

    alignments = pysam.AlignmentFile(input_file, 'rb')

    with open(output_file, 'w+') as fasta:

        for read in alignments:

            if not read.is_secondary and not read.is_supplementary and not read.is_unmapped and read.seq is not None:

                name = read.query_name
                # equivalent to soft-clip length
                start = read.query_alignment_start

                # softclip longer than 100bp
                if start > SC:
                    seq = read.seq[(start-SC) :(start+ALN)]

                # softclip smaller than 100bp
                elif start <= SC:
                    seq = read.seq[:(start+ALN)]

                fasta.write(f'>{name}\n{seq}\n')

In [ ]:
# Handler function
def extract_softclip_end_handler(ID):
    
    input_file = f'{path}/{ID}/{ID}-transcriptome_sorted.bam'
    output_file= f'{path}/{ID}/{ID}-five_prime_softclip[SC=100|ALN=20].fa'

    # run script
    extract_softclip_end(input_file, output_file, SC=100, ALN=20)

    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(extract_softclip_end_handler, runs)

## 5.b. SL Search (1st Pass - on 5' soft-clip end)

In [ ]:
SL_ALIGN_PARAMS = {'match': 1,
                   'mismatch': -1,
                   'gap_open': 2,    # penalty to create a gap
                   'gap_extend': 1}  # penalty to extend a gap (must have created before)

In [ ]:
# For each read, all known SL1 and SL2 sequences are matched against their 5' soft-clip sequence.
# A match is returned if the alignment score is higher or equal to 70% (sensitivity parameter) of the sequence length
# ex: a 10bp sequence will be accepted if the returned score is higher of equal at 7.

# If no match is returned, the evaluated SL sequence is shortened (on the 5' side) and the match is performed again.
# ex: sequence 'AAATTGTGTGTGT' returned no match, we then search 'ATTGTGTGTGT'

# The process is repeated until we reach a minimal sequence of 7bp. If no match is found, the SL is considered not found.
# If various SL sequences return the same score they are equally accepted

def search_spliced_leaders(input_file, output_file, sensitivity=0.7, offset=0):

    seq_file = open(f'{path}/ref/SL_sequences.fasta')
    SPLICELEADERS = {record.id: str(record.seq) for record in SeqIO.parse(seq_file, "fasta")}

    reads = {}

    for record in SeqIO.parse(input_file, "fasta"):

        ref = len(record.seq)
        
        aln_scores = {}
        position = {}
        distance = {}

        for sl_name in SPLICELEADERS:

            sl_seq = SPLICELEADERS[sl_name]

            sl_length = len(sl_seq)

            aln = semi_global_alignment(str(record.seq), sl_seq, params=SL_ALIGN_PARAMS)
            score = aln.score

            if score < sl_length * sensitivity:

                for pos in reversed(range(7, sl_length, 1)):

                    aln = semi_global_alignment(str(record.seq), sl_seq[-pos:], params=SL_ALIGN_PARAMS)
                    score = aln.score

                    if score < sensitivity * pos:
                        continue
                    else:
                        aln_scores[sl_name] = score
                        position[sl_name] = pos
                        distance[sl_name] = ref - (int(aln.end_query) + 1)
                        break
            else:
                aln_scores[sl_name] = score
                position[sl_name] = sl_length
                distance[sl_name] = ref - (int(aln.end_query) + 1)

        if len(aln_scores) > 0:

            #### Get best % match SL
            top_score = max(aln_scores.values())
            best_matches = [sl for sl, value in aln_scores.items() if value == top_score]

            if len(best_matches) == 1:

                sl_found = best_matches[0]
                distance = distance[sl_found]
                reads[record.id] = (sl_found, top_score, distance)

            else:

                distance = [dist for sl, dist in distance.items() if sl in best_matches]
                small_dist = min(distance)
                ix = [n for n, dist in enumerate(distance) if dist == small_dist]
                
                closest_match = [sl for n, sl in enumerate(best_matches) if n in ix]

                sl_found = ' / '.join(closest_match)

                reads[record.id] = (sl_found, top_score, small_dist)


    final = pd.DataFrame.from_dict(reads, orient='index')
    final.columns = ['SL','score','distance_to_start']
    
    # account for bases aligned at the beginning
    final['distance_to_start'] = -(final['distance_to_start'] - offset)
    
    final.index.name = 'read'
    final.to_csv(output_file, sep='\t', index=True)

In [ ]:
# Handler function
def search_spliced_leaders_handler(ID):
    
    input_file = f'{path}/{ID}-five_prime_softclip[SC=100|ALN=20].fa'
    output_file= f'{path}/{ID}/{ID}-SL_search.tsv'

    # run script
    search_spliced_leaders(input_file, output_file, sensitivity=0.7, offset=20)

    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(search_spliced_leaders_handler, runs)

In [ ]:
# set new dataframe
sl_result = pd.DataFrame()

for ID in runs:
    
    # open SL search result as dataframe
    df = pd.read_csv(f'{path}/{ID}/{ID}-SL_search.tsv', sep='\t')
    
    # append to new dataframe
    sl_result = pd.concat([sl_result, df], axis=0)

In [ ]:
# Returns a binary result (FOUND / NOT FOUND) based on the detection of a SL with high confidence or not
# Robust SL are defined based on their alignment score (score of 10 or higher)
# Or based on how close they are found to the start of the alignment

def robust_SL(SL, score, dist):
    
    if SL:
        if score > 9 or dist > -3:
            return 'FOUND'
        else:
            return 'NOT FOUND'
    else:
        return 'NOT FOUND'

In [ ]:
def robust_variant(SL, score, dist):

    if SL:
        
        # any match which scored 10 or plus
        if score > 9 :
            return 'FOUND'
        
        # matchs which score 8 or 9 IF immediatily near the ATG
        else:
            if score > 7 and dist > -3 :
                return 'FOUND'
            else:
                return 'NOT FOUND'
    else:
        return 'NOT FOUND'

In [ ]:
# Find robust SL match
sl_result['ROBUST_SL'] = sl_result.apply(lambda x: robust_SL(SL=x['SL'], score=x['score'], dist=x['distance_to_start']), axis=1)

# Find robust SL variant
sl_result['ROBUST_VARIANT'] = sl_result.apply(lambda x: robust_variant(SL=x['SL'], score=x['score'], dist=x['distance_to_start']), axis=1)

In [ ]:
# filter for non-SL reads
reads_sl = sl_result[sl_result['ROBUST_SL']=='FOUND']
reads_not_sl = sl_result[sl_result['ROBUST_SL']=='NOT FOUND']

## 5.c. SL Search (2nd pass - on whole 5' soft-clip)

In [ ]:
# Handler function
def search_spliced_leaders_2nd_pass(ID):
    
    input_file = f'{path}/{ID}/{ID}-five_prime_softclip[SC=FULL|ALN=80].fa'
    output_file= f'{path}/{ID}/{ID}-SL_search_(2nd_pass).tsv'

    # run script
    search_spliced_leaders(input_file, output_file, sensitivity=0.7, offset=80)

    print(f'Completed run {ID}\n')

In [ ]:
# processing on all CPU cores
with Pool(4) as p:
    p.map(search_spliced_leaders_2nd_pass, runs)

In [ ]:
# set new dataframe
sl_result_2ndpass = pd.DataFrame()

for ID in runs:
    
    # open SL search result as dataframe
    df = pd.read_csv(f'{path}/{ID}/{ID}-SL_search_(2nd_pass).tsv', sep='\t')
    
    # append to new dataframe
    sl_result_2ndpass = pd.concat([sl_result_2ndpass, df], axis=0)

In [ ]:
# Find robust SL match
sl_result_2ndpass['ROBUST_SL'] = sl_result_2ndpass.apply(lambda x: 'FOUND' if x['SL']==x['SL'] and x['score'] >= 12 else 'NOT FOUND', axis=1)

# Find robust SL variant
sl_result_2ndpass['ROBUST_VARIANT'] = 'NOT FOUND'

In [ ]:
# take new SL reads
reads_no_sl = list(reads_not_sl['read'])
sl_found_2ndpass = sl_result_2ndpass[(sl_result_2ndpass['ROBUST_SL']=='FOUND') & (sl_result_2ndpass['read'].isin(reads_no_sl))]

In [ ]:
# then merge back
sl_result.set_index('read', inplace=True)
sl_result.update(sl_found_2ndpass.set_index('read'))
sl_result = sl_result.reset_index()

sl_result.columns = ['read','SL', 'SL_score', 'SL_distance', 'ROBUST_SL_FOUND', 'VARIANT_SL_FOUND'] 

In [ ]:
# Merge SL result dataframe with dataset
dataset_SSP_SL = dataset_SSP.merge(sl_result, on='read', how='left')

In [ ]:
# Save result
dataset_SSP_SL.to_csv(f'{path}/dataset_+SSP+SL.tsv', sep='\t', index=None)

---

<br>

# 6. Sequence search: Endogenous hairpins (Mimic)

This section of the notebook corresponds to the code used for searching hairpin sequences (referred as **hairpin mimic**) sequences in non-SL reads.


**a.** Definition of the algorithm used for identifying hairpin mimics.<br>
**b.** Processing result of the search algorithm and integration to the dataset table.<br>
**c.** Determination of SL and hairpin mimic events in each gene.<br>

## 6.a hairpin search algorithm

In [ ]:
HAIRPIN_ALIGN_PARAMS = {'match': 1,
                        'mismatch': 0,
                        'gap_open': 2,    # penalty to create a gap
                        'gap_extend': 1}  # penalty to extend a gap (must have created before)

In [ ]:
def hairpin_search(input_file, output_file, length_s1=12, length_s2=12, offset_value=0):
    
    read = []
    scores = []
    first_stem = []
    second_stem = []
    loop_scores = []
    
    for record in SeqIO.parse(input_file, "fasta"):
        
        # set read's constant 
        seq = record.seq
        seq_length = len(seq)
        min_index = 50
        offset = offset_value - seq_length 

        # initialize score & distance & stem regions
        hairpin_found = False
        best_score = 0
        last_best_n = 0
        s1 = None
        s2 = None
        loop_size = None

        # start recursive search
        # from last position to first positions
        for n in reversed(range(min_index, seq_length+1, 2)):

            # determine loop space available for computing loop
            available_loop = n - min_index
            max_loop = 10 if available_loop > 10 else available_loop
            
            for loop in reversed(range(0, max_loop, 1)):
                
                # generate 2 regions
                end_s2 = n
                start_s2 = end_s2 - length_s2
                stem2 = str(seq[start_s2:end_s2])

                end_s1 = start_s2 - 1 - loop
                start_s1 = end_s1 - length_s1
                stem1 = str(Seq(seq[start_s1:end_s1]).reverse_complement())
                
                # perform semi-global alignment and retrieve score
                aln = semi_global_alignment(stem1, stem2, params=HAIRPIN_ALIGN_PARAMS)
                score = aln.score

                if score > best_score:

                    best_score = score
                    s1 = (start_s1+offset, end_s1+offset)
                    s2 = (start_s2+offset, end_s2+offset)
                    loop_size = loop

                    if score >= 10:
                        hairpin_found = True
                        continue # we give a chance to find a better loop

            if hairpin_found:
                break # we do not test further regions

        # after recursive search
        read.append(record.id)
        scores.append(best_score)
        first_stem.append(s1)
        second_stem.append(s2)
        loop_scores.append(loop_size)     
    
    result = pd.DataFrame(dict(read=read, HAIRPIN_score=scores, HAIRPIN_stem1=first_stem, HAIRPIN_stem2=second_stem, HAIRPIN_loop=loop_scores))

    result.to_csv(output_file, sep='\t', index=None)
                       

In [ ]:
def hairpin_search_handler(ID):
    
    input_file=f'{path}/{ID}/{ID}-five_prime_softclip[SC=FULL|ALN=80].fa'
    output_file=f'{path}/{ID}/{ID}-HAIRPIN_search.tsv'
                                  
    # search hairpins
    hairpin_search(input_file, output_file, length_s1=12, length_s2=12, offset_value=80)
    
    print(f'Completed run {ID}\n')

In [ ]:
with Pool(4) as p:
    p.map(hairpin_search_handler, runs)

## 6.b. Processing the result

In [ ]:
df_list = []
for ID in runs:
    df = pd.read_csv(f'{path}/{ID}/{ID}-HAIRPIN_search.tsv', sep='\t')
    df_list.append(df)
    
hairpin_result = pd.concat(df_list)

# Merge SL result dataframe with dataset
dataset_SSP_SL_hairpin = dataset_SSP_SL.merge(hairpin_result, on='read', how='left')

In [ ]:
def hairpin_found(read_orientation, sl_found, hairpin_score):
    
    # hairpin accepted only on antisense reads for which we could not detect an SL sequence
    if read_orientation == 'antisense' and sl_found != 'FOUND' and hairpin_score >= 10:
        return 'FOUND' 
    else:
        return 'NOT FOUND'

In [ ]:
# Determine if there is an hairpin found or not
dataset_SSP_SL_hairpin['HAIRPIN_FOUND'] = dataset_SSP_SL_hairpin.apply(lambda x: hairpin_found(x['read_orientation'], x['ROBUST_SL_FOUND'], x['HAIRPIN_score']), axis=1)

In [ ]:
# save
dataset_SSP_SL_hairpin.to_csv(f'{path}/dataset_+SSP+SL+HAIRPIN.tsv', sep='\t', index=None)

---

<br>

# 7. Processing sequence search

In [ ]:
# Compute stats per gene per start positions (%SL / %hairpin / %unidentified)

rows = []

for (gene, position), reads in dataset_SSP_SL_hairpin.groupby(['gene','corrected_genomic_start']):

    # Total of reads at a given position
    total = len(reads)

    position = int(position)
    
    ###########################
    # Measure number of SSP  at the position 
    # (this is measured independantly from SL/hairpin/unidentified)
    ###########################
    ssp_reads = len(reads[reads['SSP_FOUND'] == 'FOUND'])
    ssp_percent = round(ssp_reads/total*100, 2)
    
    ###########################
    # Measure number of SL / hairpin / unidentified at the position
    ###########################
    sl_reads = len(reads[reads['ROBUST_SL_FOUND'] == 'FOUND'])
    sl_percent = round(sl_reads/total*100, 2)
    
    hairpin_reads = len(reads[reads['HAIRPIN_FOUND'] == 'FOUND'])
    hairpin_percent = round(hairpin_reads/total*100, 2)
    
    unidentified_reads = total - (sl_reads+hairpin_reads)
    unidentified_percent = round(unidentified_reads/total*100, 2)
    

    ###########################
    # Measure ratio SL1 / SL2
    ###########################
    
    # Sense reads from SL1_1 experiment are not used for counting SL1/SL2/Hairpin percentages
    evaluated_reads = reads[~((reads['run'] == 'SL1_1') & (reads['read_orientation'] == 'antisense'))]
    evaluated = len(evaluated_reads)
    
    if evaluated > 0:
    
        # robust variants are used to measure SL1/SL2 ratio 
        # (-> subgroup of ROBUST SL for which we have a ROBUST VARIANT)
        robust_variant = evaluated_reads[evaluated_reads['VARIANT_SL_FOUND'] == 'FOUND']
        
        # count SL1 and SL2 variants
        sl1 = len(robust_variant[robust_variant['SL'].str.contains('SL1')])
        sl2 = len(robust_variant[~(robust_variant['SL'].str.contains('SL1'))])
        
        # measure SL2/SL1 ratio
        sl2_ratio = sl2/(sl1+sl2) if (sl1+sl2)>0 else None
        
    else:
        sl1 = 0
        sl2 = 0
        sl2_ratio = None
        
    # Put numbers into a dictionnary
    row = {'gene':gene, 'position':position, 'total':total, 'SSP':ssp_reads, '%SSP':ssp_percent,
           'SL':sl_reads, 'HAIRPIN':hairpin_reads, 'UNIDENTIFIED':unidentified_reads,
           '%SL':sl_percent, '%HAIRPIN':hairpin_percent, '%UNIDENTIFIED':unidentified_percent,
           'SL1_variants':sl1, 'SL2_variants':sl2, 'SL2_ratio':sl2_ratio }
    
    rows.append(row)
    
# create dataframe
result = pd.DataFrame(rows)
result = result.sort_values(['gene','position'])

# save table
result.to_csv(f'{path}/start_positions_stats.tsv', sep='\t', index=None)